### Zadanie 1

Napisz funkcję (najlepiej przedefiniować `ToString`), która dla zadanego wyrażenia w języku programowania C# zwróci łańcuch znaków w postaci ONP. Przykładowo:

dla wyrażenia `x = a - b * c`, gdzie `a`, `b`, `c` to klasy typu `OnpExpression`, a `-`, `*` to operatory, funkcja `x.ToString()` zwróci "a b - c *" (brak priorytetów dla operacji i nawiasów).

Klasa `OnpExpression` powinna przechowywać swoje wartości tak, aby uprościć metodzie `ToString` zwracanie wyniku.

In [12]:
public abstract class OnpExpression {
    public abstract string Interpret();

    public static OnpExpression operator +(OnpExpression left, OnpExpression right) {
        return new NonTerminalExpression(left, right, "+");
    }

    public static OnpExpression operator -(OnpExpression left, OnpExpression right) {
        return new NonTerminalExpression(left, right, "-");
    }

    public static OnpExpression operator *(OnpExpression left, OnpExpression right) {
        return new NonTerminalExpression(left, right, "*");
    }

    public static OnpExpression operator /(OnpExpression left, OnpExpression right) {
        return new NonTerminalExpression(left, right, "/");
    }

    public override string ToString() {
        return Interpret();
    }

}

public class TerminalExpression : OnpExpression {
     string Value;

    public TerminalExpression(string value) {
        Value = value;
    }

    public override string Interpret() {
        return Value;
    }

    public override string ToString() {
        return Value;
    }
}

public class NonTerminalExpression : OnpExpression {
    protected OnpExpression Left;
    protected OnpExpression Right;
    protected string Operator;

    public NonTerminalExpression(OnpExpression left, OnpExpression right, string o) {
        Left = left;
        Right = right;
        Operator = o;
    }

    public override string Interpret() {
        return Left.Interpret() + " " + Right.Interpret() + " " + Operator;
    }
}

TerminalExpression a = new TerminalExpression("a");
TerminalExpression b = new TerminalExpression("b");
TerminalExpression c = new TerminalExpression("c");

OnpExpression x = a - b * c;

Console.WriteLine(x.ToString());

a b c * -



### Zadanie 2

Napisz operator dla `<` i `>`, który sprawdza, czy suma elementów listy jest większa czy mniejsza od drugiej listy. Przykładowo dla `a = [1,2,3,4]` i `b = [20,30]` `a < b` powinno zwrócić wartość `true`.

In [5]:
class CustomList<T> 
{

    List<T> list = new List<T>();

    public static bool operator <(CustomList<T> list1, CustomList<T> list2)
    {
        return (list1.GetSum() < list2.GetSum()) ? true : false;
    }
    public static bool operator >(CustomList<T> list1, CustomList<T> list2)
    {
        return (list1.GetSum() > list2.GetSum()) ? true : false;
    }

    public void Add(T item) { 
        list.Add(item);
    }

    public void Remove(T item) { 
        list.Remove(item);
    }

    public int Count() { 
        return list.Count;
    }

    public int GetSum() { 
        int sum = 0;
        foreach (var item in list)
        {
            sum += Convert.ToInt32(item);
        }
        return sum;
    }

}


CustomList<int> list1 = new CustomList<int>();
CustomList<int> list2 = new CustomList<int>();

list1.Add(1);
list1.Add(2);

list2.Add(3);
list2.Add(4);

Console.WriteLine(list1 > list2);
Console.WriteLine(list1 < list2);


False
True




### Zadanie 3

Dana jest klasa `Student` przechowująca dane typu: numer indeksu, wiek, płeć, rok studiów, semestr; klasa `Degree` przechowująca dane: przedmiot, ocenę, rok zaliczenia, semestr. Używając *LINQ*. 

* Połącz dane z klasy `Student` i `Degree`.
* Wyświetl studentów, których wiek jest większy, niż średnia dla studentów na roku.
* Wyświetl studentów, których średnia ocen jest większa niż pozostałych studentów na roku.


In [8]:
class Student(int IndexNumber, int Age, string Sex, int Year, int Semester)
{
    public int IndexNumber { get; set; } = IndexNumber;
    public int Age { get; set; } = Age;
    public string Sex { get; set; } = Sex;
    public int Year { get; set; } = Year;
    public int Semester { get; set; } = Semester;
}

class Degree(string Subject, int Grade, int Year, int Semester)
{
    public string Subject { get; set; } = Subject;
    public int Grade { get; set; } = Grade;
    public int Year { get; set; } = Year;
    public int Semester { get; set; } = Semester;
}

List<Student> students = new List<Student> {
    new Student(1, 20, "Male", 1, 1),
    new Student(2, 21, "Female", 1, 1),
};

List<Degree> degrees = new List<Degree> {
    new Degree("CS", 5, 2024, 1),
    new Degree("CS", 2, 2024, 1),
};

// Merge data from both lists
var mergedData =        from student in students
                        join degree in degrees
                        // join on the only common field
                        on new { student.Semester } equals new { degree.Semester }
                        select new { student.IndexNumber, student.Age, student.Sex, degree.Subject, degree.Grade, degree.Year, degree.Semester };

foreach (var item in mergedData)
{
    Console.WriteLine(item);
}

// * Wyświetl studentów, których wiek jest większy, niż średnia dla studentów na roku.
var studentsWithAgeGreaterThanAvg = from student in students
                                    // Calculating average (https://learn.microsoft.com/pl-pl/dotnet/api/system.linq.enumerable.average?view=net-8.0)
                                    where student.Age > students.Average(student => student.Age)
                                    select new { student.IndexNumber, student.Age, student.Sex };

foreach (var item in studentsWithAgeGreaterThanAvg)
{
    Console.WriteLine(item);
}

// * Avareage age for students on year
var studentsAvgAgeOnYear =  from student in students
                            group student by student.Year into groupedByYear
                            select new { Year = groupedByYear.Key, AverageAge = groupedByYear.Average(s => s.Age) };

foreach (var item in studentsAvgAgeOnYear)
{
    Console.WriteLine(item);
}


// * Wyświetl studentów, których średnia ocen jest większa niż pozostałych studentów na roku.    
var studentsWithHigherAvgGrade =    from student in students
                                    join degree in degrees
                                    on new { student.Year, student.Semester } equals new { degree.Year, degree.Semester }
                                    group degree by new { student.Year, student.Semester } into groupedByYear
                                    select new { groupedByYear.Key.Year, groupedByYear.Key.Semester, AverageGrade = groupedByYear.Average(grades => grades.Grade) };

foreach (var item in studentsWithHigherAvgGrade)
{
    Console.WriteLine(item);
}

{ IndexNumber = 1, Age = 20, Sex = Male, Subject = CS, Grade = 5, Year = 2024, Semester = 1 }
{ IndexNumber = 1, Age = 20, Sex = Male, Subject = CS, Grade = 2, Year = 2024, Semester = 1 }
{ IndexNumber = 2, Age = 21, Sex = Female, Subject = CS, Grade = 5, Year = 2024, Semester = 1 }
{ IndexNumber = 2, Age = 21, Sex = Female, Subject = CS, Grade = 2, Year = 2024, Semester = 1 }
{ IndexNumber = 2, Age = 21, Sex = Female }
{ Year = 1, AverageAge = 20.5 }
